In [1]:
pip install langchain-groq


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name= "llama3-70b-8192"
)
response = llm.invoke("Who is the father of computer science and AI ?")
print(response.content)

The father of computer science is widely considered to be Alan Turing (1912-1954), a British mathematician, computer scientist, logician, and philosopher. Turing made significant contributions to the development of computer science, artificial intelligence, and cryptography.

Turing's work laid the foundation for modern computer science, including:

1. **The Turing Machine**: A theoretical model for a computer that could simulate human computation, which is still studied today.
2. **The Turing Test**: A measure of a machine's ability to exhibit intelligent behavior equivalent to, or indistinguishable from, that of a human.
3. **Algorithm design**: Turing developed the concept of the algorithm, which is a set of instructions used to solve a problem.
4. **Computer programming**: Turing wrote the first computer program, which was used to crack the German Enigma code during World War II.

As for the father of Artificial Intelligence (AI), there are several candidates, but the most commonly

In [ ]:
#using webbase loader from langchain document

In [5]:
pip install langchain-community


Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.metacareers.com/jobs/1070147800777577/")
page_data = loader.load().pop().page_content
print(page_data) 


USER_AGENT environment variable not set, consider setting it to identify your requests.




Data Scientist, Product Analytics | Meta Careers

















Data Scientist, Product AnalyticsSunnyvale, CA • Los Angeles, CA • Remote, US • Bellevue, WA • Austin, TX • Menlo Park, CA • Seattle, WA • Burlingame, CA • Washington, DC • New York, NY • San Francisco, CA • Full Time+ 10 more•Full Time - HideMetaData & AnalyticsData ScienceApply to this jobAs a Data Scientist at Meta, you will shape the future of people-facing and business-facing products we build across our entire family of applications (Facebook, Instagram, Messenger, WhatsApp, Oculus). By applying your technical skills, analytical mindset, and product intuition to one of the richest data sets in the world, you will help define the experiences we build for billions of people and hundreds of millions of businesses around the world. You will collaborate on a wide array of product and business problems with a wide-range of cross-functional partners across Product, Engineering, Research, Data Engineering, Marketing, Sal

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser 

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Data Scientist, Product Analytics',
  'experience': 'A minimum of 6 years of work experience in analytics (minimum of 4 years with a Ph.D.)',
  'skills': ['SQL',
   'Python',
   'R',
   'Mathematics',
   'Statistics',
   'Data querying languages',
   'Scripting languages',
   'Statistical/mathematical software'],
  'description': 'As a Data Scientist at Meta, you will shape the future of people-facing and business-facing products we build across our entire family of applications (Facebook, Instagram, Messenger, WhatsApp, Oculus). By applying your technical skills, analytical mindset, and product intuition to one of the richest data sets in the world, you will help define the experiences we build for billions of people and hundreds of millions of businesses around the world.'}]

In [9]:
type(json_res)  ##next step I have to prepare a chromadb

list

In [10]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install hf_xet

Note: you may need to restart the kernel to use updated packages.


In [12]:

# pip install langchain-community chromadb sentence-transformers pypdf uuid

import uuid # You already have this imported later, but good to keep here
import chromadb # You already have this imported later, but good to keep here
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# 1. Loading the resume document
file_path = "app/ManiJ_Resume.pdf"
try:
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} page(s) from {file_path}")
except Exception as e:
    print(f"Error loading document: {e}")
    documents = [] # Ensure documents is empty if loading fails

# 2. Splitting the document into chunks ( 1 page for RAG)
splits = []
if documents:
    # Adjusting chunk_size and chunk_overlap as needed for resume
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
    splits = text_splitter.split_documents(documents)
    print(f"Split the resume into {len(splits)} chunks.")
else:
    print("No resume loaded to split.")

# 3. Creating embeddings using a common sentence transformer model
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
print("Initialized embedding model.")

# 4. Initializing and Populating the ChromaDB for the resume
# Use a different persistence directory or collection name than your portfolio DB if it exists
persist_directory = "chroma_db_resume"
collection_name = "resume_collection" 

# Using Chroma.from_documents creates the collection and adds the splits
# This will overwrite if you run it multiple times with the same persist_directory and collection_name
if splits:
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory=persist_directory,
        collection_name=collection_name
    )
    print(f"Stored {len(splits)} resume chunks in ChromaDB collection '{collection_name}' at {persist_directory}")

    # You will now use 'vectorstore' to query your resume content

else:
    print("No resume chunks to store in ChromaDB.")


Loaded 1 page(s) from ManiJ_Resume.pdf
Split the resume into 12 chunks.


/var/folders/l9/rmqb54914qb0hxzg0pmw90tm0000gn/T/ipykernel_47904/244743731.py:32: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/mjillepalli/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Initialized embedding model.
Stored 12 resume chunks in ChromaDB collection 'resume_collection' at chroma_db_resume


In [14]:

# Import necessary libraries (assuming llm is already defined, e.g., from langchain)
from langchain_core.prompts import PromptTemplate

email_prompt_template = """
You are an AI assistant that helps write personalized cold emails for job applications.
Your goal is to write a concise and compelling email highlighting the applicant's relevant experience
based on the provided job description, snippets from their resume, and relevant links.

Here is the job description information:
Role: {job_role}
Skills Required: {job_skills}
Job Description Snippet: {job_description_snippet}

Here are relevant snippets from the applicant's resume:
{resume_snippets}

Write a cold email from the applicant to the hiring manager for this role.
Focus on connecting the applicant's resume highlights and the content/examples from the links to the job requirements.
Keep the email professional and to the point.
Include a clear subject line.

Subject: Application for {job_role} - [Your Name]

Dear Hiring Manager,

[Write the email body here, referencing the job requirements, resume snippets, and relevant links]

Sincerely,
Mani chandana J
 Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
"""

email_prompt = PromptTemplate.from_template(email_prompt_template)

# Assume 'llm' is your initialized language model (e.g., from langchain)
# llm: BaseLanguageModel # Example type hint

# Create a chain to combine the prompt and the language model
# Use a simple string output parser if your LLM returns a complex object
# email_chain = email_prompt | llm | StrOutputParser()
email_chain = email_prompt | llm # Assuming llm directly returns a string or has a simple .content attribute

generated_emails = [] # Keep this list to store emails if you need them later





# Format the links for the prompt (e.g., as a bulleted list)
formatted_links = "\n".join([f"- {link}" for link in applicant_relevant_links])
# If the list is empty, provide a default message
if not formatted_links:
    formatted_links = "No relevant links provided."
# ----------------------------------------------


# Iterate through each job posting in the json_res list
for job in json_res:
    job_role = job.get('role', 'Unknown Role')
    # Ensure job_skills is a string, joining if it's a list
    job_skills_list = job.get('skills', [])
    job_skills = ", ".join(job_skills_list) if isinstance(job_skills_list, list) else str(job_skills_list)


    job_description_snippet = job.get('description', 'No description provided.')

    print(f"Generating email for job: {job_role}")

    # Ensure the 'skills' key exists and is not empty before querying resume
    relevant_resume_text = "No relevant resume snippets found."
    # Use the processed job_skills for the query
    if job_skills:
        query_text = job_skills # Use the joined string of skills for the query

        # Query the vectorstore to get the top relevant chunks from the resume
        # Adjust k as needed to get enough context for the email
        documents = vectorstore.similarity_search(query_text, k=3) # Getting top 3 chunks

        # Concatenate the content of the relevant resume chunks
        if documents:
            relevant_resume_text = "\n---\n".join([doc.page_content for doc in documents])
        else:
             relevant_resume_text = "No highly relevant resume snippets found based on skills."


    # Prepare the input for the email generation chain
    email_input = {
        'job_role': job_role,
        'job_skills': job_skills, # Pass the formatted skills string
        'job_description_snippet': job_description_snippet,
        'resume_snippets': relevant_resume_text,
        'relevant_links': formatted_links # Pass the formatted list of links
    }

    # Invoke the chain to generate the email
    try:
        # Assuming llm.invoke returns an object with a .content attribute
        generated_email_content = email_chain.invoke(email_input).content
        generated_emails.append({'job_role': job_role, 'email': generated_email_content})

        # Print the generated email to the notebook output
        print("\n--- Generated Email ---")
        print(generated_email_content)
        print("-----------------------\n")

    except Exception as e:
        print(f"Error generating email for {job_role}: {e}")
        generated_emails.append({'job_role': job_role, 'email': f"Failed to generate email: {e}"})

    print("-" * 30) # Separator for clarity between jobs

# 'generated_emails' list still contains dictionaries with job role and generated email content
# You can access it here if needed after the loop finishes


Generating email for job: Data Scientist, Product Analytics

--- Generated Email ---
Here is a cold email from the applicant to the hiring manager for the Data Scientist, Product Analytics role:

Subject: Application for Data Scientist, Product Analytics - Mani Chandana J

Dear Hiring Manager,

I'm excited to apply for the Data Scientist, Product Analytics role at Meta, where I can leverage my technical skills, analytical mindset, and product intuition to shape the future of people-facing and business-facing products across your entire family of applications.

With a strong background in SQL, Python, and R, I'm confident in my ability to extract insights from complex data sets and drive product decisions. My experience in data analysis, engineering, and visualization, as well as my proficiency in TensorFlow, PyTorch, and scikit-learn, aligns well with the requirements of this role. I'm impressed by Meta's commitment to using data to inform product decisions and believe my skills would 